In [ ]:
import ipywidgets as widgets
import numpy as np
# from scipy.stats import skewnorm 
import matplotlib.pyplot as plt
from math import erf
import pandas as pd
import scipy
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score
import scipy.stats as stats

In [ ]:

# p_dir = 'r'
# n_dir = 'l'

positives=widgets.FloatRangeSlider(
    value=[0., 1.],
    min=0.0,
    max=1.0,
    step=0.01,
    description='P_Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

negatives=widgets.FloatRangeSlider(
    value=[0., 1.],
    min=0.0,
    max=1.0,
    step=0.01,
    description='N_Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

p_pow = widgets.FloatLogSlider(
    value=0,
    base=10,
    min=0, 
    max=2, 
    step=0.1,
    description='P_Power'
)

n_pow = widgets.FloatLogSlider(
    value=0,
    base=10,
    min=0, 
    max=2, 
    step=0.1,
    description='N_Power'
)

p_num = widgets.Dropdown(
    options=[10, 50, 100, 1000, 10000],
    value=100,
    description='P_Number:',
    disabled=False,
)

n_num = widgets.Dropdown(
    options=[10, 50, 100, 1000, 10000],
    value=100,
    description='N_Number:',
    disabled=False,
)

p_dir = widgets.Dropdown(
    options=['r', 'l'],
    value='r',
    description='P_Bias:',
    disabled=False,
)

n_dir = widgets.Dropdown(
    options=['r', 'l'],
    value='l',
    description='N_Bias:',
    disabled=False,
)

# An HBox lays out its children horizontally
# ui = widgets.HBox([negatives, positives,p_pow])

items = [negatives, positives, n_pow, p_pow, n_num, p_num, n_dir, p_dir]
ui = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"))

def plot(p, n, p_pow, n_pow, p_num, n_num, p_dir, n_dir):
    np.random.seed(1)

    mu, sigma = 0.5, 0.1 # mean and standard deviation

    fig, axs = plt.subplots(2, 2,figsize=(10,10))
    # obs_pos = np.random.normal(mu, sigma, n_pos)
    pos_probs = np.random.rand(p_num)
    if p_dir =='l':
      cons = 0.
      coef = +1
    elif p_dir =='r':
      cons = 1.
      coef = -1  
    pos_probs = (cons+coef*pos_probs**p_pow)*(p[1]-p[0])+p[0]

    axs[1,1].plot(pos_probs,np.zeros((p_num)),'o',  c='green', alpha=0.1)
    y, x, _  = axs[1,1].hist(pos_probs,20,color='lightgreen', alpha=0.5)
    axs[1,1].set_xlim([-0.1,1.1])
    axs[1,1].set_ylim(bottom=-0.1*(np.max(y)-np.min(y)))
    axs[1,1].set_title("Distribution of Positives")

    neg_probs = np.random.rand(n_num)
    if n_dir =='l':
      cons = 0.
      coef = +1
    elif n_dir =='r':
      cons = 1.
      coef = -1  
    neg_probs = (cons+coef*neg_probs**n_pow)*(n[1]-n[0])+n[0]

    axs[1,0].plot(neg_probs,np.zeros((n_num)),'o', c='red', alpha=0.1)
    y, x, _  = axs[1,0].hist(neg_probs,20,color='pink', alpha=0.5)
    axs[1,0].set_xlim([-0.1,1.1])
    axs[1,0].set_ylim(bottom=-0.1*(np.max(y)-np.min(y)))
    axs[1,0].set_title("Distribution of Negatives")    

    testy = np.array(np.zeros(n_num).tolist() + np.ones(p_num).tolist())
    probs = np.array(neg_probs.tolist() + pos_probs.tolist())
    fper, tper, thresholds = roc_curve(testy, probs) 
    roc_auc = roc_auc_score(testy, probs)
    ar = 2*roc_auc-1.
    axs[0,0].plot(fper, tper, color='orange', label='ROC - AUC={:0.2f} - AR={:0.2f}'.format(roc_auc,ar))
    axs[0,0].plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    axs[0,0].set_xlabel('False Positive Rate')
    axs[0,0].set_ylabel('True Positive Rate')
    axs[0,0].set_title('Receiver Operating Characteristic (ROC) Curve')
    axs[0,0].legend()
    axs[0,0].set_ylim([-0.1,1.1])    

    p, r, thresholds = precision_recall_curve(testy, probs) 
    axs[0,1].plot(r, p, color='orange')
    axs[0,1].set_xlabel('Recall')
    axs[0,1].set_ylabel('Precision')
    axs[0,1].set_title('Precision - Recall Curve')
    axs[0,1].set_ylim([-0.1,1.1])

    plt.show()

out = widgets.interactive_output(plot, {'p': positives, 'n': negatives, 'p_pow' : p_pow, 'n_pow' : n_pow,
                                        'p_num' : p_num, 'n_num' : n_num, 'p_dir' : p_dir, 'n_dir' : n_dir})

display(out, ui)

Output()

GridBox(children=(FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='N_Range:', max=1.0,…